In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc 

In [6]:
table = pd.read_table("resultats-par-niveau-burvot-t1-france-entiere.txt")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 9: invalid continuation byte